In [2]:
import sys
sys.path.append("../")

In [4]:
from app import db
from db import DeadManPackage, Connectivity
import pandas as pd
from  analytics.connection import connection, _connection_for_device, connection_timeseries, connection_data_per_day
from analytics.sensors import get_sensor_data
from datetime import timedelta, date, time
from utils.date import start_of_day, end_of_day
import numpy as np
from utils.interval import find_intervals
from analytics.mouse import get_mouse_data_raw, get_mouse_data_aggregated
from clustering.clustering import get_input_data, plot_distributions, input_data_clustering, cluster_palette
import seaborn as sns
from sklearn.model_selection import train_test_split
from matplotlib.figure import Figure

from bokeh.models import ColumnDataSource
from bokeh.plotting import show, output_notebook, figure
from bokeh.palettes import Category20c
from bokeh import palettes
from bokeh import layouts
from bokeh.transform import cumsum
output_notebook()

Loading BokehJS ...

In [13]:
from db import InstructionPackage
ip = InstructionPackage

In [ ]:
def get_settings(device: str):
    ip = InstructionPackage
    query = db.session.query(ip) \
        .filter(ip.device == device) \
        .filter(ip.instruction == "SAVE")
    data = query.all()
    return data

In [49]:
query = db.session.query(ip.instruction, ip.target, ip.value, ip.timestamp) \
    .filter(ip.device == "PTL_RD_AT_001") \
    .filter(ip.timestamp >= d1) \
    .filter(ip.instruction == "SAVE")

In [50]:
device = "PTL_RD_AT_003"
d1 = date(2020, 1, 1)
d2 = date.today() - timedelta(days=1)

In [51]:
data = query.all()

In [52]:
data

[('SAVE',
  'SETTINGS',
  "{'vertical': {'wall': {'cct': 3370, 'intensities': [82.0, 82.0, 65.6, 16.4, 0.0, 0.0, 0.0, 0.0, 16.4, 65.6, 82.0, 82.0]}, 'table': {'cct': 5100, 'intensities': [45.24, 45.24, 43.62, 14.38, 12.76, 12.76, 12.76, 12.76, 14.38, 43.62, 45.24, 45.24]}}, 'horizontal': {'wall': {'cct': 4000, 'intensities': [15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0]}, 'table': {'cct': 3900, 'intensities': [40.0, 40.0, 40.0, 40.0, 40.0, 40.0, 40.0, 40.0, 40.0, 40.0, 40.0, 40.0]}}}",
  datetime.datetime(2020, 2, 26, 8, 42, 57, 125890)),
 ('SAVE',
  'SETTINGS',
  "{'vertical': {'wall': {'cct': 3370, 'intensities': [82.0, 82.0, 65.6, 16.4, 0.0, 0.0, 0.0, 0.0, 16.4, 65.6, 82.0, 82.0]}, 'table': {'cct': 5100, 'intensities': [45.24, 45.24, 43.62, 14.38, 12.76, 12.76, 12.76, 12.76, 14.38, 43.62, 45.24, 45.24]}}, 'horizontal': {'wall': {'cct': 4000, 'intensities': [30.0, 30.0, 24.0, 6.0, 0.0, 0.0, 0.0, 0.0, 6.0, 24.0, 30.0, 30.0]}, 'table': {'cct': 3900, 'intensit

In [37]:
from ast import literal_eval

In [45]:
literal_eval(data[4][2])

{'vertical': {'wall': {'cct': 4000,
   'intensities': [10.5,
    10.5,
    9.1,
    4.9,
    3.5,
    3.5,
    3.5,
    3.5,
    4.9,
    9.1,
    10.5,
    10.5]},
  'table': {'cct': 5100,
   'intensities': [17.5,
    17.5,
    17.5,
    17.5,
    17.5,
    17.5,
    17.5,
    17.5,
    17.5,
    17.5,
    17.5,
    17.5]}},
 'horizontal': {'wall': {'cct': 4000,
   'intensities': [0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0]},
  'table': {'cct': 4000,
   'intensities': [0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0]}}}

In [6]:
def total_minutes(t) -> int:
    return t.hour * 60 + t.minute

In [7]:
cluster_data = input_data_clustering(device, d1)
cluster_data['date'] = cluster_data.index.date
cluster_data['time'] = cluster_data.index.time
cluster_data.loc[:, 'minutes'] = cluster_data.time.apply(total_minutes)

[2020-06-04 11:22:28] DEBUG(root) in db: db_cached: name=clustering.clustering.input_data_clustering, parameter_digest=95d416ce22768fbdf09e0af12fd00bda7d71a60a078de99977f8503b211dc530
[2020-06-04 11:22:28] DEBUG(root) in db: db_cached: found cached value.
[2020-06-04 11:22:28] DEBUG(root) in db: db_cached: cached value expired. now=2020-06-04, cache=2020-05-27
[2020-06-04 11:22:28] DEBUG(root) in db: db_cached: Calculate data.
[2020-06-04 11:23:05] DEBUG(root) in db: db_cached: update cached data.
/home/huss/.virtualenvs/dashboard/lib/python3.6/site-packages/sqlalchemy/engine/default.py:590: Warning: (1300, "Invalid utf8 character string: '800495'")
  cursor.execute(statement, parameters)
[2020-06-04 11:23:05] DEBUG(root) in db: db_cached: return data.


In [8]:
cluster_data

,cluster,date,time,minutes
timestamp,,,,
2020-02-10 12:13:00,2,2020-02-10,12:13:00,733
2020-02-10 12:14:00,2,2020-02-10,12:14:00,734
2020-02-10 12:15:00,2,2020-02-10,12:15:00,735
2020-02-10 12:16:00,2,2020-02-10,12:16:00,736
2020-02-10 12:17:00,2,2020-02-10,12:17:00,737
...,...,...,...,...
2020-03-19 16:58:00,2,2020-03-19,16:58:00,1018
2020-03-19 16:59:00,2,2020-03-19,16:59:00,1019
2020-03-19 17:00:00,2,2020-03-19,17:00:00,1020


In [46]:
def color_to_hex(c):
    r, g, b = c
    r = int(r * 255)
    g = int(g * 255)
    b = int(b * 255)
    return f"#{r:02x}{g:02x}{b:02x}"

num_clusters = len(cluster_data.cluster.unique())
palette = [color_to_hex(c) for c in cluster_palette(num_clusters)]
palette

['#023eff', '#ff7c00', '#1ac938', '#e8000b', '#8b2be2']

In [48]:
cluster_data.loc[:, 'color'] = cluster_data.cluster.apply(lambda c: palette[c])

In [49]:
cluster_data

,cluster,date,time,minutes,color
timestamp,,,,,
2020-02-10 12:13:00,4,2020-02-10,12:13:00,733,#8b2be2
2020-02-10 12:14:00,4,2020-02-10,12:14:00,734,#8b2be2
2020-02-10 12:15:00,4,2020-02-10,12:15:00,735,#8b2be2
2020-02-10 12:16:00,4,2020-02-10,12:16:00,736,#8b2be2
2020-02-10 12:17:00,4,2020-02-10,12:17:00,737,#8b2be2
...,...,...,...,...,...
2020-03-26 11:59:00,0,2020-03-26,11:59:00,719,#023eff
2020-03-26 12:00:00,0,2020-03-26,12:00:00,720,#023eff
2020-03-26 12:01:00,0,2020-03-26,12:01:00,721,#023eff


In [50]:
source = ColumnDataSource(cluster_data)

In [61]:
cluster_data.minutes.max()

1439

In [57]:
def index_to_time(index: int):
    hours = index // 60
    minutes = index % 60
    return time(hours, minutes)

In [72]:
{m: str(index_to_time(m)) for m in minutes}

{0: '00:00:00',
 1: '00:01:00',
 2: '00:02:00',
 3: '00:03:00',
 4: '00:04:00',
 5: '00:05:00',
 6: '00:06:00',
 7: '00:07:00',
 8: '00:08:00',
 9: '00:09:00',
 10: '00:10:00',
 11: '00:11:00',
 12: '00:12:00',
 13: '00:13:00',
 14: '00:14:00',
 15: '00:15:00',
 16: '00:16:00',
 17: '00:17:00',
 18: '00:18:00',
 19: '00:19:00',
 20: '00:20:00',
 21: '00:21:00',
 22: '00:22:00',
 23: '00:23:00',
 24: '00:24:00',
 25: '00:25:00',
 26: '00:26:00',
 27: '00:27:00',
 28: '00:28:00',
 29: '00:29:00',
 30: '00:30:00',
 31: '00:31:00',
 32: '00:32:00',
 33: '00:33:00',
 34: '00:34:00',
 35: '00:35:00',
 36: '00:36:00',
 37: '00:37:00',
 38: '00:38:00',
 39: '00:39:00',
 40: '00:40:00',
 41: '00:41:00',
 42: '00:42:00',
 43: '00:43:00',
 44: '00:44:00',
 45: '00:45:00',
 46: '00:46:00',
 47: '00:47:00',
 48: '00:48:00',
 49: '00:49:00',
 50: '00:50:00',
 51: '00:51:00',
 52: '00:52:00',
 53: '00:53:00',
 54: '00:54:00',
 55: '00:55:00',
 56: '00:56:00',
 57: '00:57:00',
 58: '00:58:00',
 59: '0

In [76]:
x_range = 0, 60*24
y_range = d1, d2
fig = figure(plot_height=500, plot_width=1000,
                 title=f"Cluster timeline",
                 x_axis_type='linear',
                 y_axis_type='datetime',
                 x_range=x_range,
                 y_range=y_range,
                 tools="")

fig.rect(x='minutes', y='date', width=1, height=timedelta(days=1), color='color', source=source)

minutes = list(range(0, 60*24, 3*60))
fig.xaxis.ticker = minutes
fig.xaxis.major_label_overrides = {m: str(index_to_time(m)) for m in minutes}
 
show(fig)

In [77]:
fig.xaxis.ticker?

In [51]:
timedelta(days=1)

datetime.timedelta(1)

In [79]:
import bokeh.models.tickers as tickers

In [81]:
tickers.FixedTicker?

In [21]:
from analytics.instruction import get_state_data
from utils.date import start_of_day, end_of_day

interval = (start_of_day(d1), end_of_day(d2))
state_data = get_state_data(device, interval, resample_rule='1S')

In [6]:
def map_power(s):
    if s == "OFF":
        return 0
    return 1

In [11]:
power = pd.DataFrame(state_data.power.apply(map_power))

In [16]:
power.resample("1Min").mean()

,power
timestamp,
2020-02-07 10:06:00,0.000000
2020-02-07 10:07:00,0.883333
2020-02-07 10:08:00,1.000000
2020-02-07 10:09:00,0.000000
2020-02-07 10:10:00,0.000000
...,...
2020-05-14 23:55:00,0.000000
2020-05-14 23:56:00,0.000000
2020-05-14 23:57:00,0.000000


In [22]:
state_data.loc[:, 'power'] = (state_data.power == "ON").astype(int)

In [23]:
state_data

,power,scene,settings
timestamp,,,
2020-02-07 10:06:27,0,AUTO,OFF
2020-02-07 10:06:28,0,AUTO,OFF
2020-02-07 10:06:29,0,AUTO,OFF
2020-02-07 10:06:30,0,AUTO,OFF
2020-02-07 10:06:31,0,AUTO,OFF
...,...,...,...
2020-05-14 23:59:55,0,AUTO,OFF
2020-05-14 23:59:56,0,AUTO,OFF
2020-05-14 23:59:57,0,AUTO,OFF


In [25]:
state_data.power.unique()

array([0, 1])

In [3]:
from analytics.instruction import get_power

In [4]:
power_data = get_power(device, d1, resample_rule='1Min')
power_data

,power
timestamp,
2020-02-07 10:06:00,0.000000
2020-02-07 10:07:00,0.883333
2020-02-07 10:08:00,1.000000
2020-02-07 10:09:00,0.000000
2020-02-07 10:10:00,0.000000
...,...
2020-05-15 23:55:00,0.000000
2020-05-15 23:56:00,0.000000
2020-05-15 23:57:00,0.000000


In [8]:
from bokeh.palettes import Magma256

In [9]:
len(Magma256)

256

In [3]:
from ajax_plots import PlotPowerTimeline

In [4]:
pp = PlotPowerTimeline(plot_parameters={device: "PTL_RD_AT_003"})

In [5]:
data = pp._fetch()

Fetch data for None
